In [28]:
#importing necessary libraries 
import numpy as np
import math

In [29]:
'''
Implement the feedforward and backpropagation learning algorithm for multiple perceptrons in Python for the question provided in the attached image.

Initialize the weights and biases randomly.

Implement the forward pass.

Compute the loss between the predicted output 
and the actual output using an appropriate loss function.

Compute the gradients of the loss function with respect to the weights 
and biases using the chain rule.

Update the weights and biases.

Iterate over multiple times (epochs), performing forward propagation, 
loss calculation, backpropagation, and parameter updates in each iteration till convergence.

'''

'\nImplement the feedforward and backpropagation learning algorithm for multiple perceptrons in Python for the question provided in the attached image.\n\nInitialize the weights and biases randomly.\n\nImplement the forward pass.\n\nCompute the loss between the predicted output \nand the actual output using an appropriate loss function.\n\nCompute the gradients of the loss function with respect to the weights \nand biases using the chain rule.\n\nUpdate the weights and biases.\n\nIterate over multiple times (epochs), performing forward propagation, \nloss calculation, backpropagation, and parameter updates in each iteration till convergence.\n\n'

In [30]:
#first im going to perform one iteration with the weight values given in class to check the correctness of the code: 

w1 = np.array([[0.5, 1.5, 0.8],[0.8,0.2,-1.6]])
w2 = np.array([[0.9, -1.7, 1.6],[1.2,2.1,-0.2]])
x = np.array([1, 0.7, 1.2])
target = np.array([1, 0])
w1, w2, x

(array([[ 0.5,  1.5,  0.8],
        [ 0.8,  0.2, -1.6]]),
 array([[ 0.9, -1.7,  1.6],
        [ 1.2,  2.1, -0.2]]),
 array([1. , 0.7, 1.2]))

In [31]:
#defining the sigmoid activation function 
def activation_function(num):
    return 1 / (1 + (math.e ** (-num)))

In [32]:
#implement forward pass
def forward_pass(w1, w2, x):
    layer_0_layer_1 = np.dot(x, w1.T)  
    #print(f"** x:{x} , w1:{w1}, x_shape:{x.shape}, w1_shape:{w1.shape}, mul:{layer_0_layer_1}")
    #print(f"mul:{layer_0_layer_1}")
    layer_1_output = [activation_function(val) for val in layer_0_layer_1]
    layer_1_output = np.insert(layer_1_output, 0, 1)
    layer_1_layer_2 = np.dot(layer_1_output , w2.T)
    layer_2_output = [activation_function(val) for val in layer_1_layer_2]
    return layer_2_output, layer_1_output

output_2, output_1 = forward_pass(w1, w2, x)
print(f"Output of nodes FORMAT : O_nodePosition_layerValue, e.g. O_0_1 means Output of 0th node in 1st layer\n")
print(f"Output of nodes in layer 1\nO_0_1:{output_1[0]}\tO_1_1:{output_1[1]}\tO_2_1:{output_1[2]}")
print(f"Output of nodes in layer 2\nO_0_2:{output_2[0]} \tO_1_2:{output_2[1]}")

Output of nodes FORMAT : O_nodePosition_layerValue, e.g. O_0_1 means Output of 0th node in 1st layer

Output of nodes in layer 1
O_0_1:1.0	O_1_1:0.9248398905178734	O_2_1:0.2728917836588705
Output of nodes in layer 2
O_0_2:0.441370708075126 	O_1_2:0.9563777409741487


In [33]:
#this function computes derivative of error at the output layer for all nodes
def compute_error_output_layer(output, target):
    error_at_layer = np.array([])
    for i in range(0,len(output)):
        val = (output[i] - target[i]) * output[i] * (1 - output[i])
        error_at_layer = np.append(error_at_layer, val)
    return error_at_layer

In [34]:
def backprop(w1,w2,x,output_2, output_1, target):
    learning_rate = 0.5

    error_at_2 = compute_error_output_layer(output_2, target)

    #computing derivatives of weights with respect to error between layer 2 and layer 1
    #derivatives with respect to X_0_1 , X_1_1, X_2_1

    #defining a matrix of size (2,3) to store all the derivatives for each weight 
    dE_dw_2_1 = np.zeros_like(w2)
    for i in range(len(error_at_2)):
        for j in range(len(output_1)):
            dE_dw_2_1[i][j] = error_at_2[i] * output_1[j]

    # UPDATING WEIGHTS BETWEEN LAYER 2 AND LAYER 1
    for i in range(len(w2)):
        for j in range(len(w2[0])):
            w2[i][j] -= learning_rate * dE_dw_2_1[i][j]

    
    #computing derivatives of weights with respect to error between layer 1 and layer 0
            
    #derivative of error at layer 1:  ,basically backpropagating error from layer 2 to layer 1 
    error_node_1 = (error_at_2[0] * w2[0][1] + error_at_2[1] * w2[1][1]) * output_1[1] * (1 - output_1[1])
    error_node_2 = (error_at_2[0] * w2[0][2] + error_at_2[1] * w2[1][2]) * output_1[2] * (1 - output_1[2])

    error_at_1 = np.array([error_node_1, error_node_2])

    #derivative of error with respect to weights between nodes in layer 1 and layer 0
    dE_dw_1_0 = np.zeros_like(w1)
    for i in range(len(error_at_1)):
        for j in range(len(x)):
            dE_dw_1_0[i][j] = error_at_1[i] * x[j]

    #updating the weights between layer 0 and layer 1
    w1 -= learning_rate * dE_dw_1_0

    return w2, w1

w2_new, w1_new = backprop(w1,w2,x,output_2,output_1,target)
print(f"W1 after first epoch : {w1_new}")
print(f"W2 after first epoch : {w2_new}")

W1 after first epoch : [[ 0.48928025  1.49249617  0.7871363 ]
 [ 0.8229341   0.21605387 -1.57247908]]
W2 after first epoch : [[ 0.96886855 -1.63630762  1.61879366]
 [ 1.18005027  2.08154969 -0.20544412]]


In [35]:
def mse_error_function(output, target):
    error = 0.0
    for i in range(len(output)):
        error += ((output[i] - target[i]) ** 2)
    return error

In [36]:
def training_loop(x, target, w1, w2, epochs, tolerance):
    number_of_epochs_needed = 0
    for epoch in range(epochs):
        number_of_epochs_needed += 1
        output_2, output_1 = forward_pass(w1, w2, x)
        if mse_error_function(output_2, target) < tolerance:
            break
        else:
            w2, w1 = backprop(w1,w2,x,output_2, output_1, target)
    return w1,w2, number_of_epochs_needed, output_2

In [37]:
#lets find the optimised weights and bias with starting weight as the ones taught in class
w1 = np.array([[0.5, 1.5, 0.8],[0.8,0.2,-1.6]])
w2 = np.array([[0.9, -1.7, 1.6],[1.2,2.1,-0.2]])
x = np.array([1, 0.7, 1.2])
target = np.array([1, 0])
epochs = 2000
tolerance = 0.0005
final_w1, final_w2, number_of_epochs, actual_output = training_loop(x, target, w1, w2, epochs, tolerance)

print(f"W1 optimised : {final_w1}")
print(f"W2 optimised : {final_w2}")
print(f"number of epochs : {number_of_epochs}")
print(f"Actual Output : {actual_output}")
print(f"Target Output : {target}")
print(f"MSE : {mse_error_function(actual_output, target)}")

W1 optimised : [[ 0.08929562  1.21250694  0.30715475]
 [ 1.8756602   0.95296214 -0.30920775]]
W2 optimised : [[ 2.34102724 -0.46644573  2.43300646]
 [-1.84235253 -0.45820664 -2.11974796]]
number of epochs : 1573
Actual Output : [0.9846022106089666, 0.016209925753329742]
Target Output : [1 0]
MSE : 0.0004998536110590842


In [38]:
#now lets randomly choose weights and train 
np.random.seed(42)


W1 = np.random.uniform(0,3, size = (2,3))
W2 = np.random.uniform(0,3, size=(2,3))
X = np.array([1, 0.7, 1.2])
Target = np.array([1, 0])
epochs = 2000
tolerance = 0.0005

final_w1, final_w2, number_of_epochs, actual_output = training_loop(X, Target, W1, W2, epochs, tolerance)

print(f"W1 optimised : {final_w1}")
print(f"W2 optimised : {final_w2}")
print(f"number of epochs : {number_of_epochs}")
print(f"Actual Output : {actual_output}")
print(f"Target Output : {Target}")
print(f"MSE : {mse_error_function(actual_output, Target)}")

W1 optimised : [[ 1.13906699  2.86295556  2.21451778]
 [ 1.36599669  0.16707079 -0.04799095]]
W2 optimised : [[ 0.2682638   2.69225504  1.87995681]
 [-1.01868421 -3.07137003  0.18702629]]
number of epochs : 1205
Actual Output : [0.988659557208424, 0.019264997579689906]
Target Output : [1 0]
MSE : 0.0004997457744544655


As we can see, there are multiple W1 and W2 matrices that could be initiated which would give the same target values 
Based on the tolerance 0.0005, the teacher given weights and randomly selected weights usually converge around 1500 iterations